In [1]:
# organizations 우리가 필요한 필드 전부 넣음.
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize 


In [2]:
field_ids =  ['acquirer_identifier', 'categories', 'category_groups', 'closed_on', 'company_type', 'created_at', 'delisted_on', 'description', 'entity_def_id', 'equity_funding_total', 'exited_on', 'facet_ids', 'founded_on', 'founder_identifiers', 'funding_stage', 'funding_total', 'funds_total', 'hub_tags', 'identifier', 'investor_identifiers', 'investor_stage', 'investor_type', 'ipo_status', 'last_equity_funding_total', 'last_equity_funding_type', 'last_funding_at', 'last_funding_total', 'last_funding_type', 'last_key_employee_change_date', 'layout_id', 'legal_name', 'linkedin', 'listed_stock_symbol', 'location_group_identifiers', 'location_identifiers', 'name', 'num_acquisitions', 'num_articles', 'num_current_advisor_positions', 'num_current_positions', 'num_diversity_spotlight_investments', 'num_employees_enum', 'num_enrollments', 'num_event_appearances', 'num_exits', 'num_exits_ipo', 'num_founders', 'num_funding_rounds', 'num_funds', 'num_investments', 'num_investors', 'num_lead_investments', 'num_lead_investors', 'num_past_positions', 'num_portfolio_organizations', 'num_sub_organizations', 'operating_status', 'owner_identifier', 'permalink', 'rank_delta_d30', 'rank_delta_d7', 'rank_delta_d90', 'rank_org', 'rank_principal', 'revenue_range', 'short_description', 'status', 'stock_symbol', 'uuid', 'valuation', 'valuation_date', 'went_public_on']
len(field_ids)

userkey = {"user_key":"b1130d42de5b1c31f60949e441cdaa9a"}

query = {
"field_ids": [
"identifier",
"location_identifiers",
"short_description",
"founded_on", "last_funding_type"
],
"limit": 1000
}

def company_count(query):
    r = requests.post("https://api.crunchbase.com/api/v4/searches/organizations", params = userkey , json = query)
    result = json.loads(r.text)
    total_companies = result["count"]
    return total_companies

def url_extraction(query):
    global raw
    r = requests.post("https://api.crunchbase.com/api/v4/searches/organizations", params = userkey , json = query)
    result = json.loads(r.text)
    normalized_raw = json_normalize(result['entities'])
    raw = raw.append(normalized_raw,ignore_index=True)

In [ ]:
raw=pd.DataFrame()
comp_count = company_count(query)
data_acq = 0

# data_acq
while data_acq < comp_count:
    print(raw)
    if data_acq != 0:
        last_uuid = raw.uuid[len(raw.uuid)-1]
        query["after_id"] = last_uuid
        url_extraction(query)
        data_acq = len(raw.uuid)
    else:
        if "after_id" in query:
            query = query.pop("after_id")
            url_extraction(query)
            data_acq = len(raw.uuid)
        else:
            url_extraction(query)
            data_acq = len(raw.uuid)


In [ ]:
# pickle 로 저장  (df안에 리스트형태가 있을까봐 혹시.)

import pickle
path = 해당 컴터의 path
with open(path,'wb') as fw:
  pickle.dump(raw,fw)

# pickle load 확인 
with open(path,'rb') as fr:
  raw_test = pickle.load(fr)
raw_test # 나오면 잘된거.
